In [ ]:
!pip install groq
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
import json
import re
import logging
from datetime import date, timedelta, datetime
from groq import Groq

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set up environment
api_key = "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"
client = Groq(api_key=api_key)

In [ ]:
def parse_relative_date(relative_date_str):
    today = date.today()
    if relative_date_str.lower() == "last quarter":
        quarter = (today.month - 1) // 3 + 1
        start_month = 3 * (quarter - 2) + 1
        start_date = date(today.year, start_month, 1)
        end_date = start_date + timedelta(days=89)
    elif relative_date_str.lower() == "previous month":
        start_date = date(today.year, today.month - 1, 1)
        end_date = date(today.year, today.month, 1) - timedelta(days=1)
    else:
        start_date = today - timedelta(days=365)
        end_date = today
    return start_date.isoformat(), end_date.isoformat()

def process_query(query, history=[]):
    """
    Processes a user query and returns a JSON object.
    """
    #LLM extract information
    prompt = f"""
    Extract the following information from the user query related to company performance metrics:

    - Entities: The company names mentioned in the query.
    - Parameters: The performance metrics mentioned in the query.
    - Start Date: The start date of the time period (if not mentioned, assume today's date minus one year).
    - End Date: The end date of the time period (if not mentioned, assume today's date).

    User query: {query}

    Output the extracted information in JSON format EXACTLY like this:
    {{
      "entities": ["company_name1", "company_name2"],
      "parameters": ["metric_name1", "metric_name2"],
      "start_date": "YYYY-MM-DD",
      "end_date": "YYYY-MM-DD"
    }}

    Do not include any extra text or information in the response.
    """
    messages = [{"role": "user", "content": prompt}]
    if history:
        messages = history[-5:] + messages

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        temperature=0,
        max_tokens=1024,
        top_p=1,
    )


    try:
        extracted_info = json.loads(completion.choices[0].message.content)
    except json.JSONDecodeError:
        extracted_info = {
            "entities": [],
            "parameters": [],
            "start_date": None,
            "end_date": None
        }


    today = date.today()
    if not extracted_info.get("start_date"):
        extracted_info["start_date"] = (today - timedelta(days=365)).isoformat()
    if not extracted_info.get("end_date"):
        extracted_info["end_date"] = today.isoformat()


    if re.match(r"last quarter|previous month", extracted_info.get("start_date", ""), re.IGNORECASE):
        extracted_info["start_date"], extracted_info["end_date"] = parse_relative_date(extracted_info["start_date"])


    output_json = {
        "request": {
            "query": query
        },
        "response": {
            "data": []
        }
    }

    for entity in extracted_info["entities"]:
        for parameter in extracted_info["parameters"]:
            output_json["response"]["data"].append({
                "entity": entity,
                "parameter": parameter,
                "time_period": {
                    "start_date": extracted_info["start_date"],
                    "end_date": extracted_info["end_date"]
                }
            })

    return output_json



In [ ]:
def main():
    history = []
    while True:
        query = input("Enter your query (or 'exit' to quit): ")
        if query.lower() == 'exit':
            break

        response_data = process_query(query, history)
        history.append({"role": "user", "content": query})
        history.append({"role": "assistant", "content": json.dumps(response_data)})

        if not response_data:
            print("Error: Unable to extract information from the query.")
            continue

        results_json = json.dumps(response_data, indent=4)


        print(results_json)

if __name__ == "__main__":
    main()

Enter your query (or 'exit' to quit): gmv for amazon
{
    "request": {
        "query": "gmv for amazon"
    },
    "response": {
        "data": [
            {
                "entity": "Amazon",
                "parameter": "gmv",
                "time_period": {
                    "start_date": "2023-12-24",
                    "end_date": "2024-12-24"
                }
            }
        ]
    }
}
Enter your query (or 'exit' to quit): compare this with flipkart
{
    "request": {
        "query": "compare this with flipkart"
    },
    "response": {
        "data": [
            {
                "entity": "Amazon",
                "parameter": "gmv",
                "time_period": {
                    "start_date": "2023-12-24",
                    "end_date": "2024-12-24"
                }
            },
            {
                "entity": "Flipkart",
                "parameter": "gmv",
                "time_period": {
                    "start_date": "2023-12-24",
  